In [1]:
import numpy as np

from scipy.io import wavfile
import math

In [2]:
# Encoding
def rice_encoder(Sample, K):
    
    # get sign of sample
    if Sample < 0:
        Sample = - Sample
        sign_sam = "-"
    else:
        sign_sam = "+"

    # Compute M 
    M = 2**K

    # For Sample, find quotient and remainder
    q = int(Sample / M)
    r = Sample % M

    # get code word
    code_word = ""
    for i in range(q):
        # Add 1 q number of times.
        code_word += "1"
  

    # get r in binary
    
    b = K
    a = "{0:0" + str(b) + "b}"
    r_bin = a.format(r)

    # Encoded string
    encode_str = sign_sam + code_word + "0" + str(r_bin)

    return encode_str


In [3]:
# Decoding
def rice_decoder(Sample, K):
    #  separate
    sign_sam = Sample[0]
    Sample = Sample[1:]

    # Compute M 
    M = 2**K

    # Separate quotient and remainder 
    q_r = Sample.split('0', 1)

    # Get quotient 
    quotient = len(q_r[0])

    # Get remainder 
    remainder = int(q_r[1][:K], 2)

    # get encoded number 
    Sample = quotient * M + remainder

    # Change sign of the decoded bit
    if sign_sam == '-':
        Sample = -Sample

    return Sample

In [4]:
# read sound file, encode, decode, and write new wav file
def read_and_write(filepath:str):
    sr, audio_data = wavfile.read(filepath)


    # Encode sound file and save as _Enc.ex2
    index = filepath.find(".wav")
    encoded_name = filepath[:index] + "_Enc.ex2"
    K = 4 #bit number
    with open(encoded_name, 'wb') as cf:
        for i in range(len(audio_data)):
            e = rice_encoder(audio_data[i], K) + '\n'
            cf.write(e.encode())
    cf.close()

    # Decode the encoded sound file
    decoded_sound = []
    with open(encoded_name, 'rb') as cf:
        for i in cf:
            decoded_sound.append(rice_decoder(i.decode('utf8').strip(), K))
    cf.close()
    decoded_sound = np.array(decoded_sound, dtype='int16')


    # Write decoded audio file in .wav format
    idx = encoded_name.find(".ex2")
    decoded_name = encoded_name[:idx] + "Dec.wav"
    wavfile.write(decoded_name, sr, decoded_sound)

In [5]:
read_and_write('sounds/Sound1.wav')
read_and_write('sounds/Sound2.wav')

Check that the original audio file and the reconstructed file are the same.

In [18]:
import wave
import os

audio1 = wave.open('sounds/Sound1.wav','rb')
audio1_specs = audio1.getparams()
print('Sound1.wav:',audio1_specs)
print('Sound1.wav size, in bytes:',os.stat('sounds/Sound1.wav').st_size)

print('\n')

audio1_recon = wave.open('sounds/Sound1_EncDec.wav','rb')
audio1_specs_recon = audio1_recon.getparams()
print('Sound1_EncDec.wav:',audio1_specs_recon)
print('Sound1_EncDec.wav size, in bytes:',os.stat('sounds/Sound1_EncDec.wav').st_size)

print('\n')

audio2 = wave.open('sounds/Sound2.wav','rb')
audio2_specs = audio2.getparams()
print('Sound2.wav:',audio2_specs)
print('Sound2.wav size, in bytes:',os.stat('sounds/Sound2.wav').st_size)

print('\n')

audio2_recon = wave.open('sounds/Sound2_EncDec.wav','rb')
audio2_specs_recon = audio2_recon.getparams()
print('Sound2_EncDec.wav:',audio2_specs_recon)
print('Sound2_EncDec.wav size, in bytes:',os.stat('sounds/Sound2_EncDec.wav').st_size)

Sound1.wav: _wave_params(nchannels=1, sampwidth=2, framerate=44100, nframes=501022, comptype='NONE', compname='not compressed')
Sound1.wav size, in bytes: 1002088


Sound1_EncDec.wav: _wave_params(nchannels=1, sampwidth=2, framerate=44100, nframes=501022, comptype='NONE', compname='not compressed')
Sound1_EncDec.wav size, in bytes: 1002088


Sound2.wav: _wave_params(nchannels=1, sampwidth=2, framerate=44100, nframes=504000, comptype='NONE', compname='not compressed')
Sound2.wav size, in bytes: 1008044


Sound2_EncDec.wav: _wave_params(nchannels=1, sampwidth=2, framerate=44100, nframes=504000, comptype='NONE', compname='not compressed')
Sound2_EncDec.wav size, in bytes: 1008044


Files are identical. Successfully encoded and decoded using lossless compression.

|               | Original size | Rice(K = 4 bits) | Rice(k = 2 bits) | %compression(K = 4 bits) | %Compression(k = 2 bits) |
|:-------------:|:-------------:|:----------------:|:----------------:|:------------------------:|:------------------------:|
|  Sound1.wav   |     1MB       |       25MB       |       89.6MB     |        -2400 %           |            -8860 %       |
|  Sound2.wav   |     1MB       |       150MB      |       591MB      |        -14900            |            -59000 %      |
